In [2]:
import pickle
import albumentations as A
from torchvision import transforms, datasets
import torch.utils.data as data
from PIL import Image
import os
import json
from torchvision import transforms
import random
import numpy as np
import json
import torch
import pickle
from tqdm import tqdm
import cv2
import torch.nn as nn
import torch.nn.functional as F
import math

In [ ]:
!pip install albumentations

In [3]:
from tqdm import tqdm
path_to_image = {}
for i in tqdm(range(0, 23)):
    path = '/scratch/sm9669/nabird pickle/nabird/train_'+str(i*1000)+'.pkl'
    file = open(path,'rb')
    sub_data = pickle.load(file)
    path_to_image.update(sub_data)

100%|██████████| 23/23 [01:01<00:00,  2.69s/it]


In [5]:
mean = (0.5071, 0.4867, 0.4408)
std = (0.2675, 0.2565, 0.2761)
normalize = transforms.Normalize(mean=mean, std=std)

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=299, scale=(0.2, 1.)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    normalize,
])

In [16]:
def default_loader(path):
    image= path_to_image[path]
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(img)
#     return Image.open(path).convert('RGB')
    return im_pil

class NABirds(data.Dataset):
    def __init__(self, root, meta_data, split_name, im_size_resize, im_size_crop, is_train=True, meta_type='ebird_meta', transforms=None):

        # load annotations
        print('Loading annotations from: ' + os.path.basename(meta_data))
        with open(meta_data) as fp:
            da = json.load(fp)

        # only choose images that are available - ie some missing from download
        # also select the split of interest
        da = [dd for dd in da[split_name] if dd['valid_image']]

        # set up the filenames and annotations
        self.imgs = [dd['im_path'] for dd in da]
        self.classes = [dd['class_id'] for dd in da]
        self.users = [dd[meta_type]['user_id'] for dd in da]
        self.dates = [dd[meta_type]['date'] for dd in da]
        self.lon = [dd[meta_type]['lon'] for dd in da]
        self.lat = [dd[meta_type]['lat'] for dd in da]

        # print out some stats
#         print split_name
#         print '\t' + str(len(self.imgs)) + ' images'
#         print '\t' + str(len(set(self.classes))) + ' classes'

        self.root = root
        self.is_train = is_train
        self.loader = default_loader
        self.im_size_resize = im_size_resize
        self.im_size_crop = im_size_crop
        self.transforms = transforms # added to this 

        # augmentation params
        self.mu_data = [0.485, 0.456, 0.406]
        self.std_data = [0.229, 0.224, 0.225]
        self.brightness = 0.4
        self.contrast = 0.4
        self.saturation = 0.4
        self.hue = 0.25

        # augmentations
#         self.resize = transforms.Resize(self.im_size_resize)
#         self.resize_rand = transforms.RandomResizedCrop(self.im_size_crop)
#         self.center_crop = transforms.CenterCrop(self.im_size_crop)
#         self.flip_aug = transforms.RandomHorizontalFlip()
#         self.color_aug = transforms.ColorJitter(self.brightness, self.contrast, self.saturation, self.hue)
#         self.tensor_aug = transforms.ToTensor()
#         self.norm_aug = transforms.Normalize(mean=self.mu_data, std=self.std_data)
        

    def __getitem__(self, index):
        path = self.imgs[index]
        im_id = self.imgs[index]
        img = self.loader(path)
        class_id = self.classes[index]

        if self.is_train:
            #img = self.resize_rand(img)
            #img = self.flip_aug(img)
            #img = self.color_aug(img)
            #print("su")
            z = "sumit"
        else:
            img = self.resize(img)
            img = self.center_crop(img)

#         img = self.tensor_aug(img)
#         img = self.norm_aug(img)
        
        if self.transforms:
            img = [self.transforms(img), self.transforms(img)]
        
        return img, class_id

    def __len__(self):
        return len(self.imgs)

In [17]:
meta_data = '/scratch/sm9669/nabirds_with_loc_2019.json'
data_root = '/nabirds/images/'

In [18]:
train_dataset = NABirds(data_root, meta_data,
                     split_name='train', im_size_crop=299,
                     im_size_resize=342, is_train=True,transforms = train_transform)
    
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
               shuffle=True, num_workers=0, pin_memory=True)

Loading annotations from: nabirds_with_loc_2019.json


In [19]:
it = iter(train_loader)
first = next(it)
print(len(train_loader))

179


In [20]:
class SupConLoss(nn.Module):
    
    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def forward(self, features, labels=None, mask=None):
    
        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask

        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        loss = loss.view(anchor_count, batch_size).mean()

        return loss

In [22]:
from torchvision import models
class Linear_layer(nn.Module):
    def __init__(self, input_dim,output_dim,feat_dim, head='mlp'):
        super(Linear_layer, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        if head == 'linear':
            self.head = nn.Linear(dim_in, feat_dim)
        elif head == 'mlp':
            self.head = nn.Sequential(
                nn.ReLU(inplace=True),
                nn.Linear(2048, feat_dim)
            )
        else:
            raise NotImplementedError(
                'head not supported: {}'.format(head))
        
    def forward(self, x):
        feat = self.fc(x)
        feat = F.normalize(self.head(feat), dim=1)
        return feat


class SupConResNet(nn.Module):
    def __init__(self, name='resnet50', head='mlp', feat_dim=1024):
        super(SupConResNet, self).__init__()
        self.encoder = models.inception_v3(pretrained=True,aux_logits =False)
        self.encoder.fc = Linear_layer(2048, 2048,feat_dim, head)
       

    def forward(self, x):
        feat = self.encoder(x)
        return feat
    


In [24]:
model = SupConResNet()
criterion = SupConLoss(temperature=0.1)
model = model.cuda()
criterion = criterion.cuda()

In [25]:
lr =  0.5
optimizer = torch.optim.SGD(model.parameters(),
                          lr=lr,
                          momentum=0.9,
                          weight_decay=1e-4)

In [27]:
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    epoch_loss=0
    
    for idx, (images, labels) in enumerate(train_loader):
        #print(type(images[0]))
        images = torch.cat([images[0], images[1]], dim=0)
        if torch.cuda.is_available():
            images = images.cuda(non_blocking=True)
            labels = labels.cuda(non_blocking=True)
        bsz = labels.shape[0]

        # warm-up learning rate
        #warmup_learning_rate(opt, epoch, idx, len(train_loader), optimizer)

        # compute loss
        features = model(images)
        f1, f2 = torch.split(features, [bsz, bsz], dim=0)
        features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)
        #print(features.shape)
        loss = criterion(features, labels)
        epoch_loss+=loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch {} - Train Loss {:.2f}'.format(epoch + 1, epoch_loss ))
    PATH = '/scratch/sm9669/nabird pickle/model_'+str(epoch+1)+'_'+str(epoch_loss)+'_inceptionV3.pth'
    torch.save(model.state_dict(), PATH)
    #print('Training Accuracy {:.4f}'.format(train_correct_total * 100 / count))


In [28]:
for epoch in tqdm(range(0, 50)):
    #lr = lr * (0.1 ** (epoch // 50))
    #adjust_learning_rate(epoch)
    eta_min = lr * (0.1 ** 3)
    lr = eta_min + (lr - eta_min) * (
            1 + math.cos(math.pi * epoch / 50)) / 2

    train(train_loader, model, criterion, optimizer, epoch)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1 - Train Loss 1256.65


  4%|▍         | 2/50 [27:21<10:56:47, 820.98s/it]

Epoch 2 - Train Loss 1033.41


  6%|▌         | 3/50 [41:02<10:43:05, 820.98s/it]

...

Epoch 48 - Train Loss 474.53


 98%|█████████▊| 49/50 [11:30:59<13:55, 835.03s/it]

Epoch 49 - Train Loss 464.27


100%|██████████| 50/50 [11:44:42<00:00, 845.64s/it]

Epoch 50 - Train Loss 465.52


In [ ]:
PATH = '/scratch/sm9669/nabird pickle/model_4_inceptionV3.pth'

In [ ]:
!pwd

In [ ]:
torch.save(model.state_dict(), PATH)


In [ ]:
model.load_state_dict(torch.load(PATH))

In [1]:
!nvidia-smi

Wed Apr 27 03:13:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:06:00.0 Off |                    0 |
| N/A   38C    P0    53W / 250W |      0MiB / 45556MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------